In [1]:
import pickle
import pandas as pd
import numpy as np
from scipy.stats.mstats import gmean
import sklearn
import statsmodels.formula.api as sm
from statsmodels.tools import add_constant
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from sklearn.linear_model import LogisticRegression, LinearRegression
from numpy.linalg import LinAlgError
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
fpath = "/Users/ys8mz/Box Sync/Predictive Models of College Completion (VCCS)/intermediate_files"

In [2]:
df_old = pd.read_stata(fpath + "/full_data_truncated.dta")
pvcc_ind = pd.read_stata(fpath + "/pvcc_ind.dta")
df = df_old.merge(pvcc_ind, how='inner', on=['vccsid']).drop(['pvcc'],axis=1)

In [3]:
train_df = df[df.valid == 0]
test_df = df[df.valid == 1]
print(train_df.shape,test_df.shape)

(7132, 341) (837, 341)


In [6]:
## Share of the pvcc students in the full dataset:
7132/298139, 837/33115

(0.023921727784691035, 0.025275554884493432)

In [4]:
sum(train_df.grad_6years)/train_df.shape[0]

0.33524957936062816

In [4]:
sum(test_df.grad_6years)/test_df.shape[0]

0.35722819593787336

In [5]:
predictors = list(df.columns)[10:]
print(len(predictors))
impute_list_1 = set(["prop_comp_pre","cum_gpa_pre"])
impute_list_2 = set([t1+"_"+t2+str(t3) for t1 in ["term_gpa", "prop_comp", "lvl2_prop_comp", "dev_prop_comp"] for t2 in ["fa", "sp", "su"] for t3 in range(1,7,1)])
impute_list_3 = set(["cum_gpa", "lvl2_prop_comp", "dev_prop_comp", "prop_comp", "prop_comp_sd", "withdrawn_prop_comp_sd"])
impute_list_4 = set(["admrate", "gradrate", "satvr25", "satvr75", "satmt25", "satmt75", "satwr25", "satwr75"])

331


In [6]:
# degree completion rate of validation sample
sum(train_df.grad_6years)/train_df.shape[0]

0.33524957936062816

In [7]:
# degree completion rate of validation sample
sum(test_df.grad_6years)/test_df.shape[0]

0.35722819593787336

In [8]:
def impute(train, test):
    for p in impute_list_1:
        avg_p = np.nanmean(train[train.enrolled_pre == 1][p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    for p in impute_list_3:
        avg_p = np.nanmean(train[p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    for p in impute_list_2:
        suffix = p[-3:]
        avg_p = np.nanmean(train[train["enrolled_" + suffix] == 1][p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    for p in impute_list_4:
        avg_p = np.nanmean(train[train["enrolled_nsc"] == 1][p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    return train, test                    

In [9]:
train_df_new, test_df_new = impute(train_df, test_df)
X_train = train_df_new.loc[:,predictors]
y_train = train_df_new.grad_6years
X_test = test_df_new.loc[:,predictors]
y_test = test_df_new.grad_6years

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
results_dir = "C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\smaller_training_sample\\test2\\"

#### (1) Run the logistic regression model

In [11]:
try:
    lr = sm.Logit(y_train, add_constant(X_train,prepend=True)).fit()
except LinAlgError:
    lr = LogisticRegression(random_state=1234)
    lr.fit(X_train, y_train)

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1264: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


         Current function value: inf
         Iterations: 35


In [12]:
print("Logistic Regression:")
if type(lr) == LogisticRegression:
    y_test_pred_lr = list(lr.predict_proba(X_test)[:,1])
else:
    y_test_pred_lr = list(lr.predict(add_constant(X_test, prepend=True)))
print("AUC = {}".format(round(roc_auc_score(y_test, y_test_pred_lr),4)))

Logistic Regression:
AUC = 0.9015


In [14]:
# save the model object and predicted scores on the validation sample to local disk
# pickle.dump(lr, open(results_dir + "/lr.p", "wb"))
pickle.dump(list(y_test_pred_lr), open(results_dir + "y_test_pred_lr.p", "wb"))

In [17]:
def find_optimal_threshold(p,r,t):
    to_drop = np.union1d(np.where(pd.isnull(p[:-1]) == True)[0], np.where(pd.isnull(r[:-1]) == True)[0])
    to_drop = np.union1d(to_drop, np.where(pd.isnull(t) == True)[0])
    to_keep = np.setdiff1d(np.array(list(range(len(p)-1))), to_drop)
    p,r,t = p[to_keep],r[to_keep],t[to_keep]
    f1 = 2*p*r/(p+r)
    best_t = t[np.argmax(f1)]
    best_t
    return best_t

In [18]:
def cross_validation_lr(train):
    threshold_list = []
    auc_list = []
    k_fold =  StratifiedKFold(n_splits = 10, random_state = 12345, shuffle=True)
    for train_indices, test_indices in k_fold.split(train, train.grad_6years):
        train_part = train.iloc[train_indices,:]
        test_part = train.iloc[test_indices,:]
        train_part_new, test_part_new = impute(train_part, test_part)
        X_1 = train_part_new.loc[:,predictors]
        y_1 = train_part_new.grad_6years
        X_2 = test_part_new.loc[:,predictors]
        y_2 = test_part_new.grad_6years
        model = LogisticRegression(random_state=1234)
        model.fit(X_1,y_1)
        p,r,t = precision_recall_curve(y_2, model.predict_proba(X_2)[:,1])
        auc = roc_auc_score(y_2, model.predict_proba(X_2)[:,1])
        threshold_list.append(find_optimal_threshold(p,r,t))
        auc_list.append(auc)
    print(np.mean(auc_list), np.std(auc_list, ddof=1))
    return gmean(threshold_list)                                   

In [19]:
best_threshold = cross_validation_lr(train_df)

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item

0.8958626184589168 0.017761706447429047


In [20]:
best_threshold

0.38588703907426586

In [21]:
def create_confusion_matrix(y_test_pred, threshold, fname):
    cm_arr = confusion_matrix(y_test, np.where(np.array(y_test_pred) > threshold, 1, 0))
    cm_df = pd.DataFrame(cm_arr, columns=['Pred_0','Pred_1'], index=['Real_0', 'Real_1'])
    cm_df.loc[:,''] = cm_df.sum(axis=1)
    cm_df.loc['',:] = cm_df.sum(axis=0)
    print(cm_df)
    print("")
    p1 = cm_df.iloc[1,1]/cm_df.iloc[2,1]
    r1 = cm_df.iloc[1,1]/cm_df.iloc[1,2]
    p0 = cm_df.iloc[0,0]/cm_df.iloc[2,0]
    r0 = cm_df.iloc[0,0]/cm_df.iloc[0,2]    
    print("F1 score = {}".format(round(2*p1*r1/(p1+r1),4)))    
    cm_df.to_csv(results_dir + fname + ".csv")
    return p1,r1,p0,r0,round(2*p1*r1/(p1+r1),4)

In [22]:
print("F1 threshold = {}:\n".format(str(round(best_threshold,3))))
pr_lr = create_confusion_matrix(y_test_pred_lr, best_threshold, "LR1_cm1")

F1 threshold = 0.386:

        Pred_0  Pred_1       
Real_0   457.0    81.0  538.0
Real_1    72.0   227.0  299.0
         529.0   308.0  837.0

F1 score = 0.7479


In [23]:
num_of_0 = int(round((1-np.mean(train_df.grad_6years))*len(y_test)))
y_test_pred_binary = np.ones(len(y_test))
y_test_pred_binary[np.argsort(y_test_pred_lr)[:num_of_0]] = 0
alternative_threshold = y_test_pred_lr[np.argsort(y_test_pred_lr)[num_of_0]]
print("Alternative threshold = {}:\n".format(str(round(alternative_threshold,3))))
pr2_lr = create_confusion_matrix(y_test_pred_binary, best_threshold, "LR1_cm2")

Alternative threshold = 0.458:

        Pred_0  Pred_1       
Real_0   471.0    67.0  538.0
Real_1    85.0   214.0  299.0
         556.0   281.0  837.0

F1 score = 0.7379


In [24]:
precision_recall_df = pd.DataFrame([(best_threshold,)+pr_lr,(alternative_threshold,)+pr2_lr]).round(4)
precision_recall_df.index = ['F1','Same_Graduation_Rate']
precision_recall_df.columns = ['threshold','precision_1','recall_1','precision_0','recall_0','f1_score']
precision_recall_df.to_csv(results_dir + "LR1_precision_recall.csv", index=True)

#### (2) Run the OLS model

In [14]:
try:
    ols = sm.OLS(y_train, add_constant(X_train,prepend=True)).fit()
except LinAlgError:
    ols = LinearRegression()
    ols.fit(X_train, y_train)

In [15]:
print("OLS:")
if type(ols) == LinearRegression:
    y_test_pred_ols = list(ols.predict(X_test))
else:
    y_test_pred_ols = list(ols.predict(add_constant(X_test, prepend=True)))
print("AUC = {}".format(round(roc_auc_score(y_test, y_test_pred_ols),4)))

OLS:
AUC = 0.8978


In [29]:
# pickle.dump(ols, open(results_dir + "/ols.p", "wb"))
pickle.dump(list(y_test_pred_ols), open(results_dir + "y_test_pred_ols.p", "wb"))

In [31]:
def cross_validation_ols(train):
    threshold_list = []
    auc_list = []
    k_fold =  StratifiedKFold(n_splits = 10, random_state = 12345, shuffle=True)
    for train_indices, test_indices in k_fold.split(train, train.grad_6years):
        train_part = train.iloc[train_indices,:]
        test_part = train.iloc[test_indices,:]
        train_part_new, test_part_new = impute(train_part, test_part)
        X_1 = train_part_new.loc[:,predictors]
        y_1 = train_part_new.grad_6years
        X_2 = test_part_new.loc[:,predictors]
        y_2 = test_part_new.grad_6years
        model = LinearRegression()
        model.fit(X_2,y_2)
        p,r,t = precision_recall_curve(y_2, model.predict(X_2))
        auc = roc_auc_score(y_2, model.predict(X_2))
        threshold_list.append(find_optimal_threshold(p,r,t))
        auc_list.append(auc)
    print(np.mean(auc_list), np.std(auc_list, ddof=1))
    return gmean(threshold_list)   

In [32]:
best_threshold_3 = cross_validation_ols(train_df)

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

0.9716525052568394 0.009157628736364262


In [33]:
best_threshold_3

0.4704189189540865

In [34]:
print("F1 threshold = {}:\n".format(str(round(best_threshold_3,3))))
pr_ols = create_confusion_matrix(y_test_pred_ols, best_threshold_3, "OLS_cm1")

F1 threshold = 0.47:

        Pred_0  Pred_1       
Real_0   473.0    65.0  538.0
Real_1    87.0   212.0  299.0
         560.0   277.0  837.0

F1 score = 0.7361


In [35]:
num_of_0 = int(round((1-np.mean(train_df.grad_6years))*len(y_test)))
y_test_pred_binary = np.ones(len(y_test))
y_test_pred_binary[np.argsort(y_test_pred_ols)[:num_of_0]] = 0
alternative_threshold_3 = y_test_pred_ols[np.argsort(y_test_pred_ols)[num_of_0]]
print("Alternative threshold = {}:\n".format(str(round(alternative_threshold_3,3))))
pr2_ols = create_confusion_matrix(y_test_pred_binary, best_threshold_3, "OLS_cm2")

Alternative threshold = 0.466:

        Pred_0  Pred_1       
Real_0   470.0    68.0  538.0
Real_1    86.0   213.0  299.0
         556.0   281.0  837.0

F1 score = 0.7345


In [36]:
precision_recall_df_3 = pd.DataFrame([(best_threshold_3,)+pr_ols,(alternative_threshold_3,)+pr2_ols]).round(4)
precision_recall_df_3.index = ['F1','Same_Graduation_Rate']
precision_recall_df_3.columns = ['threshold','precision_1','recall_1','precision_0','recall_0','f1_score']
precision_recall_df_3.to_csv(results_dir + "OLS_precision_recall.csv", index=True)

#### (3) Compare with the base model (trained on the full training sample)

In [30]:
results_dir_new = "C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\smaller_training_sample\\test2\\comparison\\"

In [31]:
model_dir = "C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\truncated_predictors\\"
base_lr = pickle.load(open(model_dir + "lr.p", "rb"))
base_ols = pickle.load(open(model_dir + "ols.p", "rb"))

In [32]:
df_new = df_old.merge(pvcc_ind, how='inner', on=['vccsid']).drop(['pvcc'],axis=1)
test_df = df_new[df_new.valid == 1]
train_df_old = df_old[df_old.valid == 0]

In [33]:
_, test_df_new = impute(train_df_old, test_df)
X_test = test_df_new.loc[:,predictors]
y_test = test_df_new.grad_6years

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [34]:
print("Logistic Regression:")
if type(base_lr) == LogisticRegression:
    y_test_reduced_pred_lr = list(base_lr.predict_proba(X_test)[:,1])
else:
    y_test_reduced_pred_lr = list(base_lr.predict(add_constant(X_test, prepend=True)))
print("AUC = {}".format(round(roc_auc_score(y_test, y_test_reduced_pred_lr),4)))

Logistic Regression:
AUC = 0.9127


In [35]:
print("OLS:")
if type(ols) == LinearRegression:
    y_test_reduced_pred_ols = list(base_ols.predict(X_test))
else:
    y_test_reduced_pred_ols = list(base_ols.predict(add_constant(X_test, prepend=True)))
print("AUC = {}".format(round(roc_auc_score(y_test, y_test_reduced_pred_ols),4)))

OLS:
AUC = 0.9083


In [36]:
def create_confusion_matrix_new(y_test, y_test_pred, threshold, fpath, fname):
    cm_arr = confusion_matrix(y_test, np.where(np.array(y_test_pred) > threshold, 1, 0))
    cm_df = pd.DataFrame(cm_arr, columns=['Pred_0','Pred_1'], index=['Real_0', 'Real_1'])
    cm_df.loc[:,''] = cm_df.sum(axis=1)
    cm_df.loc['',:] = cm_df.sum(axis=0)
    print(cm_df)
    print("")
    p1 = cm_df.iloc[1,1]/cm_df.iloc[2,1]
    r1 = cm_df.iloc[1,1]/cm_df.iloc[1,2]
    p0 = cm_df.iloc[0,0]/cm_df.iloc[2,0]
    r0 = cm_df.iloc[0,0]/cm_df.iloc[0,2]    
    print("F1 score = {}".format(round(2*p1*r1/(p1+r1),4)))    
    cm_df.to_csv(fpath + fname + ".csv")
    return p1,r1,p0,r0,round(2*p1*r1/(p1+r1),4)

In [37]:
def create_pr(train_df, y_test, y_test_pred, best_threshold, fpath, mn):
    print("F1 threshold = {}:\n".format(str(round(best_threshold,3))))
    pr_lr = create_confusion_matrix_new(y_test, y_test_pred, best_threshold, fpath, "{}_cm1".format(mn))

    num_of_0 = int(round((1-np.mean(train_df.grad_6years))*len(y_test)))
    y_test_pred_binary = np.ones(len(y_test))
    y_test_pred_binary[np.argsort(y_test_pred)[:num_of_0]] = 0
    alternative_threshold = y_test_pred[np.argsort(y_test_pred)[num_of_0]]
    print("\n\n")
    print("Alternative threshold = {}:\n".format(str(round(alternative_threshold,3))))
    pr2_lr = create_confusion_matrix_new(y_test, y_test_pred_binary, best_threshold, fpath, "{}_cm2".format(mn))

    precision_recall_df = pd.DataFrame([(best_threshold,)+pr_lr,(alternative_threshold,)+pr2_lr]).round(4)
    precision_recall_df.index = ['F1','Same_Graduation_Rate']
    precision_recall_df.columns = ['threshold','precision_1','recall_1','precision_0','recall_0','f1_score']
    precision_recall_df.to_csv(fpath + "{}_precision_recall.csv".format(mn), index=True)

In [38]:
create_pr(train_df, y_test, y_test_reduced_pred_lr, 0.43344163040148626, results_dir_new, "LR1")

F1 threshold = 0.433:

        Pred_0  Pred_1       
Real_0   477.0    61.0  538.0
Real_1    72.0   227.0  299.0
         549.0   288.0  837.0

F1 score = 0.7734



Alternative threshold = 0.445:

        Pred_0  Pred_1       
Real_0   481.0    57.0  538.0
Real_1    75.0   224.0  299.0
         556.0   281.0  837.0

F1 score = 0.7724


In [39]:
create_pr(train_df, y_test, y_test_reduced_pred_ols, 0.46918787407941415, results_dir_new, "OLS")

F1 threshold = 0.469:

        Pred_0  Pred_1       
Real_0   485.0    53.0  538.0
Real_1    87.0   212.0  299.0
         572.0   265.0  837.0

F1 score = 0.7518



Alternative threshold = 0.45:

        Pred_0  Pred_1       
Real_0   477.0    61.0  538.0
Real_1    79.0   220.0  299.0
         556.0   281.0  837.0

F1 score = 0.7586
